# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('weather_raw_data.csv')
df

,City,Latitude,Longitude,Max Temp (F),Humidity,Cloud Coverage,Wind Speed,Country,Datetime
0,namatanai,-3.67,152.43,81.28,79.0,99,3.87,PG,1594621403
1,severo-kurilsk,50.68,156.12,50.95,84.0,98,10.83,RU,1594621403
2,cabo san lucas,22.89,-109.91,84.00,63.0,71,9.13,MX,1594621404
3,busselton,-33.65,115.33,66.00,46.0,73,18.41,AU,1594621404
4,husavik,66.04,-17.34,50.00,100.0,75,2.24,IS,1594621404
...,...,...,...,...,...,...,...,...,...
556,eyemouth,55.87,-2.09,57.99,86.0,92,4.00,GB,1594621631
557,corumba,-19.01,-57.65,72.61,72.0,13,1.72,BR,1594621631
558,yangjiang,21.85,111.97,89.40,66.0,28,8.37,CN,1594621632
559,hit,33.64,42.83,96.42,24.0,0,15.97,IQ,1594621632


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Store latitiude adn longitude in locations variable
locations = df[['Latitude', 'Longitude']]
# Store humidty data as the weight
humidity = df['Humidity']
# Configure gmap
gmaps.configure(api_key=g_key)
figure = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, point_radius=2,
                                 max_intensity=100)
                                 
# # Add heat map layer to fig with .add_layer() function
figure.add_layer(heat_layer)
# # Display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
ideal_df = df[(df['Max Temp (F)'] > 65) & 
              (df['Max Temp (F)'] < 90) & 
              (df['Humidity'] < 70) & 
              (df['Cloud Coverage'] < 5) &
              (df['Wind Speed'] < 5)]
ideal_df

,City,Latitude,Longitude,Max Temp (F),Humidity,Cloud Coverage,Wind Speed,Country,Datetime
146,mandalgovi,45.76,106.27,80.94,28.0,0,3.58,MN,1594621457
177,bayir,37.27,28.22,86.00,40.0,0,2.24,TR,1594621470
374,agadez,19.75,10.25,86.70,30.0,4,4.38,NE,1594621553
432,zverinogolovskoye,54.46,64.85,78.22,55.0,0,4.34,RU,1594621576
470,karaidel,55.83,56.91,79.79,48.0,0,3.33,RU,1594621592
552,waddan,29.16,16.14,72.16,57.0,0,4.23,LY,1594621629


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df

C:\Program Files (x86)\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Latitude,Longitude,Max Temp (F),Humidity,Cloud Coverage,Wind Speed,Country,Datetime,Hotel Name
146,mandalgovi,45.76,106.27,80.94,28.0,0,3.58,MN,1594621457,
177,bayir,37.27,28.22,86.00,40.0,0,2.24,TR,1594621470,
374,agadez,19.75,10.25,86.70,30.0,4,4.38,NE,1594621553,
432,zverinogolovskoye,54.46,64.85,78.22,55.0,0,4.34,RU,1594621576,
470,karaidel,55.83,56.91,79.79,48.0,0,3.33,RU,1594621592,
552,waddan,29.16,16.14,72.16,57.0,0,4.23,LY,1594621629,


In [48]:
params = {
    'radius': 5000,
    'types': 'lodging',
    'keyword': 'hotel',
    'location': "30.06 , -117.18",
    'key': g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
request = requests.get(base_url, params=params).json()

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = request['results'][0]['name']
    except:    
        hotel_df.loc[index, "Hotel Name"] = 'No hotel found'
    
hotel_df

C:\Program Files (x86)\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Latitude,Longitude,Max Temp (F),Humidity,Cloud Coverage,Wind Speed,Country,Datetime,Hotel Name
146,mandalgovi,45.76,106.27,80.94,28.0,0,3.58,MN,1594621457,Mandal Hotel
177,bayir,37.27,28.22,86.00,40.0,0,2.24,TR,1594621470,No hotel found
374,agadez,19.75,10.25,86.70,30.0,4,4.38,NE,1594621553,No hotel found
432,zverinogolovskoye,54.46,64.85,78.22,55.0,0,4.34,RU,1594621576,No hotel found
470,karaidel,55.83,56.91,79.79,48.0,0,3.33,RU,1594621592,Usad'ba U Al'berta
552,waddan,29.16,16.14,72.16,57.0,0,4.23,LY,1594621629,No hotel found


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [61]:
fig = gmaps.figure()
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, 
                                 info_box_content=hotel_info)

# Display figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))